In [1]:
## imports

import numpy as np
import numpy.ma as ma
import xarray as xr

import sys
sys.path.insert(0,"/home/alberta/Work/git/lupa/ratio-kinetic-energy")
import GriddedData

import matplotlib.pyplot as plt

# palette
import matplotlib.cm as mplcm

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

import glob

In [2]:
## Dataset

dir="/mnt/alberta/equipes/IGE/meom/workdir/albert/GLORYS4V1/"


In [3]:
def compute_average_ratio_KE(file_KES,file_KEL):
    ''' from a list of files containing one month of data compute variance and average in 1°x1°x1 month boxes
    '''
    data1 = xr.open_dataset(file_KES)['vomke']
    data2 = xr.open_dataset(file_KEL)['vomke']
    navlat = xr.open_dataset(file_KES)['nav_lat']
    navlon = xr.open_dataset(file_KES)['nav_lon']
    navlat2=np.array(navlat).squeeze()
    navlon2=np.array(navlon).squeeze()
    mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
    crs = GriddedData.grdCoarsener(mgrd,crs_factor=4)
#    data1_mask=np.zeros(data1.shape)
#    data2_mask=np.zeros(data2.shape)
#    for tt in np.arange(data1_mask.shape[0]):
#        truc=np.squeeze(data1[0,:,:].values)        
#        truc[np.where(mask_filt!=1)]=np.nan
#        data1_mask[tt,:,:]=truc[:,:]   
#        truc=np.squeeze(data2[0,:,:].values)        
#        truc[np.where(mask_filt!=1)]=np.nan
#        data2_mask[tt,:,:]=truc[:,:]   

    data1m=np.nanmean(data1,axis=0)
    data1c = crs.return_ravel(np.asarray(data1m))
    data1cm = np.nanmedian(data1c,axis=-3)

    data2m=np.nanmean(data2,axis=0)
    data2c = crs.return_ravel(np.asarray(data2m))
    data2cm = np.nanmedian(data2c,axis=-3)

    ratio=data1cm/data2cm
    
    latcrs=crs.return_ravel(np.asarray(navlat2))
    loncrs=crs.return_ravel(np.asarray(navlon2))
    latcrsm=np.mean(latcrs,axis=-3)
    loncrsm=np.mean(loncrs,axis=-3)

    return loncrsm,latcrsm,ratio
 

In [4]:
def compute_and_save(month):
    mm=str(month).zfill(2)
    if month < 10:
        year=2013
    else:
        year=2012
    filt_files_month_KES='GLORYS4V1NATL_1dAV_'+str(year)+str(mm)+'_KES.nc'
    filt_files_month_KEL='GLORYS4V1NATL_1dAV_'+str(year)+str(mm)+'_KEL.nc'
    tf_files_month_KES=dir+filt_files_month_KES
    tf_files_month_KEL=dir+filt_files_month_KEL
    loncrsm, latcrsm, ratio=compute_average_ratio_KE(tf_files_month_KES,tf_files_month_KEL)
    ratio_da = xr.DataArray(ratio,dims=['xcrsm','ycrsm'],name="ratio_KE_boxcar")
    lat_crsm = xr.DataArray(latcrsm,dims=['xcrsm','ycrsm'],name="latcrsm")
    lon_crsm = xr.DataArray(loncrsm,dims=['xcrsm','ycrsm'],name="loncrsm")
    ratio_da.to_dataset().to_netcdf(path=dir+'/ratio_KE_boxaverage_y'+str(year)+'m'+str(mm)+'.nc',mode='w',engine='scipy')
    lat_crsm.to_dataset().to_netcdf(path=dir+'/ratio_KE_boxaverage_y'+str(year)+'m'+str(mm)+'.nc',mode='a',engine='scipy')
    lon_crsm.to_dataset().to_netcdf(path=dir+'/ratio_KE_boxaverage_y'+str(year)+'m'+str(mm)+'.nc',mode='a',engine='scipy')


In [5]:
compute_and_save(3)

/home/alberta/anaconda2/envs/lupa/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/alberta/anaconda2/envs/lupa/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:769: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [6]:
compute_and_save(9)